In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch

from configs.ve.cifar10_ncsnpp_deep_continuous import get_config
from score_inverse.models.utils import create_model
from score_inverse.models.ema import ExponentialMovingAverage

from score_inverse.sde import get_sde
from score_inverse.datasets.scalers import get_data_inverse_scaler, get_data_scaler
from score_inverse.sampling import get_corrector, get_predictor
from score_inverse.sampling.inverse import get_pc_inverse_solver

from score_inverse.tasks.denoise import DenoiseTask
from score_inverse.datasets.cifar10 import CIFAR10

from PIL import Image
import numpy as np

import os

os.chdir("..")
print("New Working Directory ", os.getcwd())

In [ ]:
dataset = CIFAR10(train=True)

In [ ]:
config = get_config()
config.model.num_scales = 100  # Number of discretisation steps
config.eval.batch_size = 4  # Number of samples per generation

inverse_task = DenoiseTask((3, 32, 32), noise_type='shot', severity=1).to(config.device)

ckpt_path = "checkpoints/ve/cifar10_ncsnpp_deep_continuous/checkpoint_12.pth"
loaded_state = torch.load(ckpt_path, map_location=config.device)

score_model = create_model(config)
# Still need to load the base model state since non-trainable params aren't covered by EMA
score_model.load_state_dict(loaded_state["model"], strict=False)

# Replace trainable model params with EMA params
ema = ExponentialMovingAverage(score_model.parameters(), decay=config.model.ema_rate)
ema.load_state_dict(loaded_state["ema"])
ema.copy_to(score_model.parameters())

In [ ]:
def display_img(im, scale=10):
    w, h = im.size
    display(im.resize((scale * w, scale * h), Image.NEAREST))


def display_sample(sample, scale=10, shape=None):
    samples = np.clip(sample.permute(0, 2, 3, 1).cpu().numpy() * 255.0, 0, 255).astype(
        np.uint8
    )
    if shape is None:
        shape = (
            -1,
            config.data.image_size,
            config.data.image_size,
            config.data.num_channels,
        )
    samples = samples.reshape(shape)
    im = Image.fromarray(samples[0])
    display_img(im, scale)

In [ ]:
x = dataset[13]
x = x[None, :].to(config.device)

y = inverse_task.forward(x)

display_sample(x)
display_sample(y)

In [ ]:
scaler = get_data_scaler(config)

inverse_scaler = get_data_inverse_scaler(config)
sde, sampling_eps = get_sde(config)

sampling_shape = (
    config.eval.batch_size,
    config.data.num_channels,
    config.data.image_size,
    config.data.image_size,
)
predictor = get_predictor(config.sampling.predictor.lower())
corrector = get_corrector(config.sampling.corrector.lower())
sampling_fn = get_pc_inverse_solver(
    sde=sde,
    shape=sampling_shape,
    predictor=predictor,
    corrector=corrector,
    inverse_scaler=inverse_scaler,
    snr=config.sampling.snr,
    n_steps=config.sampling.n_steps_each,
    probability_flow=config.sampling.probability_flow,
    continuous=config.training.continuous,
    denoise=config.sampling.noise_removal,
    eps=sampling_eps,
    device=config.device,
    inverse_task=inverse_task,
    lambda_=0.05,
)

In [ ]:
sample, n = sampling_fn(score_model, y.to(config.device))

In [ ]:
display_sample(y)
display_sample(sample)
display_sample(x)